#### Oppgave 1

### a)

In [1]:
import xarray as xr
from scipy.interpolate import RectBivariateSpline
import numpy as np
from matplotlib import pyplot as plt
from time import time
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import random
from interpolator import Interpolator


In [2]:
def f_first(x,t,epsilon, w):
    """""
    function calculates function f used in velocity equation.
    :param x: array
    :param t: array
    :param epsilon: float
    :param w: float
    return array
    """""
    return epsilon*np.sin(w*t)*x**2 + (1-2*epsilon*np.sin(w*t))*x
def del_f(x,t,epsilon, w):
    """""
    function calculates function f used in velocity equation.
    :param x: array
    :param t: array
    :param epsilon: float
    :param w: float
    return array
    """""
    return epsilon*np.sin(w*t)*2*x + (1-2*epsilon*np.sin(w*t))

def velocity_x(x: np.ndarray, y: np.ndarray, t: np.ndarray , A: float, epsilon: float, w: float):
    """""
    function calculates velocity in x direction.
    :param x: array
    :param y: array
    :param t: array
    :param A: float
    :param epsilon: float
    :param w: float
    return array
    """""
    return -np.pi*A*np.sin(np.pi*f_first(x,t, epsilon, w))*np.cos(np.pi*y)
def velocity_y(x: np.ndarray, y: np.ndarray, t: np.ndarray , A: float, epsilon: float, w: float):
    """""
    function calculates velocity in y direction.
    :param x: array
    :param y: array
    :param t: array
    :param A: float
    :param epsilon: float
    :param w: float
    return array
    """""
    return np.pi*A*np.cos(np.pi*f_first(x,t, epsilon, w))*np.sin(np.pi*y)*del_f(x,t, epsilon, w)
def Heun(init_pos: tuple, t_end: int, h: float, A: float, epsilon: float, w: float):
    """""
    function uses the Heun integrate method to calculate the next position for the particle
    :param init_pos: tuple
    :param t_end: int
    :param h: float
    :param A: float
    :param epsilon: float
    :param w: float
    return [array, array]
    """""
    num_particles = init_pos.shape[2]
    
    t = 0
    steps = int(t_end / h)
    x = np.empty((steps + 1, 2, num_particles))
    x[:, :, :] = init_pos

    for i in range(steps):
        k_1_x = velocity_x(x[i, 0, :], x[i, 1, :], i, A, epsilon, w)
        k_1_y = velocity_y(x[i, 0, :], x[i, 1, :], i, A, epsilon, w)

        k_2_x = velocity_x(x[i, 0, :] + k_1_x * h, x[i, 1, :] + k_1_y * h, i + h, A, epsilon, w)
        k_2_y = velocity_y(x[i, 0, :] + k_1_x * h, x[i, 1, :] + k_1_y * h, i + h, A, epsilon, w)

        x_add_x = x[i, 0, :] + (h / 2) * (k_1_x + k_2_x)
        x_add_y = x[i, 1, :] + (h / 2) * (k_1_y + k_2_y)

        x[i + 1, 0, :] = x_add_x
        x[i + 1, 1, :] = x_add_y

    return x

In [3]:
#plotter bevegelsen til partikkel ved hjelp av Heun metode, med gitte initialverdier og konstanter
A = 0.1
epsilon = 0.25
w = 1
init_pos = np.array([[[1.05, 0.5]]]).reshape(1,2,1) #startposisjon
t = 50
plt.figure(figsize=(20,10))
positions1 = Heun(init_pos, t, 1, A, epsilon, w) #gir array med posisjon for partikkelen
cmap = plt.get_cmap('Blues')
num_pos = positions1.shape[0]
for idx in range(positions1.shape[0]): #potensielt unødvendig med for-løkke, menmen
    x = positions1[:,0, :]
    y = positions1[:,1, :] 
    plt.plot(x, y, color = cmap(idx/num_pos))
plt.scatter(init_pos[:,0,:], init_pos[:,1,:], label = "timestep = 1", color = "blue", s = 99)

positions2 = Heun(init_pos, t, 0.5 , A, epsilon, w) #gir array med posisjon for partikkelen
cmap = plt.get_cmap('Greens')
num_pos = positions2.shape[0]
for idx in range(positions2.shape[0]):
    x = positions2[:,0, :]
    y = positions2[:,1, :] 
    plt.plot(x, y, color = cmap(idx/num_pos))
plt.scatter(init_pos[:,0,:], init_pos[:,1,:], label = "timestep = 0.5", color = "green", s = 99)

positions3 = Heun(init_pos, t, 0.1 , A, epsilon, w) #gir array med posisjon for partikkelen
cmap = plt.get_cmap('Reds')
num_pos = positions3.shape[0]
for idx in range(positions3.shape[0]):
    x = positions3[:,0, :]
    y = positions3[:,1, :] 
    plt.plot(x, y, color = cmap(idx/num_pos))
plt.scatter(init_pos[:,0,:], init_pos[:,1, :], label = "timestep = 0.1", color = "Red", s = 99)


plt.scatter(init_pos[:,0,:], init_pos[:,1,:], label = "Initial posistion", color = "Black", s = 100)
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Particle trajectories with different timesteps')
plt.legend()
plt.grid(True)
plt.show()


finn ut hvilken h som funker, kanskje endre på label, begrunne hvordan h vi går videre med og teste den for t = 100. kanskje endre til subplots

### b)

In [4]:
#this is a test

N_p = 100
t = 10
A = 0.1
epsilon = 0.25
w = 1
h = 0.7
x_cor = np.random.uniform(0.5, 0.6, N_p) 
y_cor = np.random.uniform(0.5, 0.6, N_p)
init_pos = np.array([[x_cor, y_cor]])
cmap = plt.get_cmap('Blues')
plt.figure(figsize = (20,10))
positions = Heun(init_pos, t, h, A, epsilon, w)
plt.scatter(positions[0,0,:], positions[0,1,:], label = "Initial posistion", color = cmap(100), s = 20)
plt.scatter(positions[-1,0,:], positions[-1,1,:], label = "Final posistion", color = "black", s = 20)
for i in range(N_p):
    plt.plot(positions[:, 0, i], positions[:, 1, i],color = cmap(i/N_p),  alpha=0.5, lw=0.5)


plt.xlabel('X')
plt.ylabel('Y')
plt.title('100 particle Trajectories')
plt.legend()
plt.grid(True)
plt.show()




### c)

In [5]:
import time
N_p = 10000
steps = 100
h = 0.01
t = 10
time_list = []
for i in range(0,N_p,steps):
    start = time.time()
    ones= np.ones(i)
    init_pos = np.array([[ones*1.05, ones*0.5]])
    Heun(np.array(init_pos), t, h,A, epsilon, w)
    end = time.time()
    time_list.append(end-start)
plt.plot(np.linspace(0, N_p, len(time_list)), time_list, label = "Heun, t = [0, 10] and h = 0.01")
plt.xlabel("Number of particles")
plt.ylabel("time [s]")
plt.title("time for simulation for increasing number of particles")
plt.legend()
plt.show()


## Oppgave 2)

### a og b)

In [6]:
def Heun_2(X: np.ndarray, days: int, h: int, f):
    

    t_end = days*24*h
    steps = int(t_end / h)
    num_particles = X.shape[1]
    x = np.empty((steps + 1, 2, num_particles))
    x[:, :, :] = X

    for i in range(0,days*24):
        k_1 = f(X, t)/111.111
        k_2 = f(X+ k_1*i*h,t + i*h)/111.111
        X_next = (X + (h*i / 2) * (k_1 + k_2)).reshape(1,2,num_particles)
        X = X_next[-1,:,:]
        x[i + 1, :, :] = X

    return x


In [7]:
file_path = "/Users/tor/Documents/Semester_4/Vitber/Norkystdata/NorKyst-800m.nc"
#file_path = (r"C:\Users\tryme\Documents\VitBer\NorKyst-800m.nc") trym sin filepath

dataset = xr.open_dataset(file_path)

f  = Interpolator(dataset = d)

h = 3600
days = 5
Np = 150
X = np.zeros((2, Np))
X[0,:] = np.random.normal(loc = 1240000, scale=10000, size = Np)
X[1,:] = np.random.normal(loc = 480000, scale=10000, size = Np)
t = 0

position = Heun_2(X, days, h, f)

fig = plt.figure(figsize=(9,6))
ax = plt.axes(projection=ccrs.NorthPolarStereo())

ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', color = '#cccccc'))

# Create projection object for converting particle positions
npstere = ccrs.Stereographic(
    central_latitude = 90,
    central_longitude = f.dataset.projection_stere.straight_vertical_longitude_from_pole,
    false_easting = f.dataset.projection_stere.false_easting,
    false_northing = f.dataset.projection_stere.false_northing,
    true_scale_latitude = f.dataset.projection_stere.standard_parallel,
)

# scatter plot positions, note the extra transform keyword
ax.scatter(position[0, 0, :], position[0, 1, :], s=1, transform=npstere, label='Initial')
for j in range(position.shape[2]):
    ax.plot(position[:, 0, j], position[:, 1, j], transform=npstere,  alpha=0.5, lw=0.5)

# Make outline a bit larger
ax.set_extent((12.5, 15, 67.5, 68.2))
# Add gridlines
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=.5, color='k', lw=.5, )

ax.legend()
plt.tight_layout()

NameError: name 'd' is not defined

lag subplots med plot fra trondheim